##CUNY - Data 612

#Final Project
##Violeta Stoyanova
##Peter Kowalchuk

#Introduction

“For those of us fortunate enough to be healthy and working from home during the outbreak of COVID-19, the possibilities for entertainment can feel both extremely confined (specifically, to our homes) and overwhelmingly endless (where in our Netflix queues to begin?) (Cohen, 2020).” Now more than ever in our socially-distanced life we depend on recommendations for movies to fill our quarantine days with entertainment. Some rely on movie experts; others look at ratings to pick flicks and cure some of the overwhelming boredom. In this regard, for our final project we will attempt to build a recommender system with the use of ‘implicit’ movie data for the model to recommend new movies to users. 

#Objective

The goal of this project is to build various models in the attempt to recommend new movies to users based on ‘implicit’ data. We will be using a dataset from kaggle.com that contains descriptions of 34,886 movies from around the world. This dataset contains a summary of the plot of each movie. The goal is to use this plot to generate recommendations based on the works present in the plot. A first implementation of the recommender will provide a recommendation based on a word or list of words provided. One can imagine how a more complex implementation of this approach could allow the user write a plot of a movie she/he would like to watch. The system then would provide recommendations based on this plot by using the list of words contained in it as input to the recommender system presented in this project.

#Methodology

For the purposes of this project we will utilize the speed and scalability of Apache Spark via Databricks. The first step will be to tokenize the data, so we are able extract some weight with the use of the ‘genre’ variable. Via TF-IDF we will first eliminate any stop words and count word frequency and the build a utility matrix. With this approach words will be equivalent to users in a conventional recommender, with the word weight provided by TF-IDF representing the ratings for each movie. Afterwards, using the techniques we have learned and applied in class, such as Item-based collaborative filtering and User-based collaborative filtering (IBCF and UBCF, respectively) and ALS (Alternating Least Squares) we will build recommender systems in order for movies to be recommended to users. 
We believe that proper evaluation is important to detect how accurate our algorithms are at recommending movies thus we will root mean square error metric (RMSE).

#Data

The dataset contains descriptions of 34,886 movies from around the world. Column descriptions are listed below:
  - Release Year - Year in which the movie was released
  - Title - Movie title
  - Origin/Ethnicity - Origin of movie (i.e. American, Bollywood, Tamil, etc.)
  - Director - Director(s)
  - Plot - Main actor and actresses
  - Genre - Movie Genre(s)
  - Wiki Page - URL of the Wikipedia page from which the plot description was scraped
  - Plot - Long form description of movie plot (WARNING: May contain spoilers!!!)

In [3]:
# File location and type
file_location = "/FileStore/tables/wiki_movie_plots_deduped.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
1901,Kansas Saloon Smashers,American,Unknown,null,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"
1901,Love by the Light of the Moon,American,Unknown,null,unknown,https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
1901,The Martyred Presidents,American,Unknown,null,unknown,https://en.wikipedia.org/wiki/The_Martyred_Presidents,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination."
In the second shot,which runs just over eight seconds long,"an assassin kneels feet of Lady Justice.""",null,null,null,null,null
1901,"Terrible Teddy, the Grizzly King",American,Unknown,null,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_the_Grizzly_King","""Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading """"His Photographer"""" and """"His Press Agent"""" respectively"
1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",null,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film),"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film),"""Alice follows a large white rabbit down a """"Rabbit-hole"""". She finds a tiny door. When she finds a bottle labeled """"Drink me"""""
She enters a kitchen,"in which there is a cook and a woman holding a baby. She persuades the woman to give her the child and takes the infant outside after the cook starts throwing things around. The baby then turns into a pig and squirms out of her grip. """"The Duchess's Cheshire Cat"""" appears and disappears a couple of times to Alice and directs her to the Mad Hatter's """"Mad Tea-Party."""" After a while",she leaves.,null,null,null,null,null

We load the data and observe how many movie plot we have

In [5]:
df.count()

Out[2]: 134113

We clean the dataframe to only include the columns we need and delete all row without movie plots

In [7]:
df=df.drop("_c0","_c2","_c3","_c4","_c6")

df=df.selectExpr("_c1 as Title","_c5 as Genre","_c7 as Plot")
df = df.filter(df.Title != "Title")
df = df.dropna()
display(df)

Title,Genre,Plot
Kansas Saloon Smashers,unknown,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"
Love by the Light of the Moon,unknown,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
The Martyred Presidents,unknown,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination."
"Terrible Teddy, the Grizzly King",unknown,"""Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading """"His Photographer"""" and """"His Press Agent"""" respectively"
Jack and the Beanstalk,unknown,"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
Alice in Wonderland,unknown,"""Alice follows a large white rabbit down a """"Rabbit-hole"""". She finds a tiny door. When she finds a bottle labeled """"Drink me"""""
The Great Train Robbery,western,"The film opens with two bandits breaking into a railroad telegraph office, where they force the operator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the locomotive's tender at the station's water tank. They then knock the operator out and tie him up. As the train stops it is boarded by the bandits‍—‌now four. Two bandits enter an express car, kill a messenger and open a box of valuables with dynamite; the others kill the fireman and force the engineer to halt the train and disconnect the locomotive. The bandits then force the passengers off the train and rifle them for their belongings. One passenger tries to escape but is instantly shot down. Carrying their loot, the bandits escape in the locomotive, later stopping in a valley where their horses had been left."
The Suburbanite,comedy,"The film is about a family who move to the suburbs, hoping for a quiet life. Things start to go wrong, and the wife gets violent and starts throwing crockery, leading to her arrest."
The Little Train Robbery,unknown,"""The opening scene shows the interior of the robbers' den. The walls are decorated with the portraits of notorious criminals and pictures il

Now we have a reduced dataframe with only movies and plot

In [9]:
df.count()

Out[4]: 69981

#TF-IDF

In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer

In [12]:
#tokenize
tokenizer = Tokenizer(inputCol="Plot", outputCol="words")
wordsData = tokenizer.transform(df)

#remove stop words
remover = StopWordsRemover(inputCol="words",outputCol="noStopWords")
wordsData = remover.transform(wordsData)
wordsData = wordsData.drop("words")
wordsData=wordsData.selectExpr("Title as Title","Genre as Genre","noStopWords as words")
display(wordsData)

Title,Genre,words
Kansas Saloon Smashers,unknown,"List(bartender, working, saloon,, serving, drinks, customers., fills, stereotypically, irish, man's, bucket, beer,, carrie, nation, followers, burst, inside., assault, irish, man,, pulling, hat, eyes, dumping, beer, head., group, begin, wrecking, bar,, smashing, fixtures,, mirrors,, breaking, cash, register., bartender, sprays, seltzer, water, nation's, face, group, policemen, appear, order, everybody, leave.[1])"
Love by the Light of the Moon,unknown,"List(moon,, painted, smiling, face, hangs, park, night., young, couple, walking, past, fence, learn, railing, look, up., moon, smiles., embrace,, moon's, smile, gets, bigger., sit, bench, tree., moon's, view, blocked,, causing, frown., last, scene,, man, fans, woman, hat, moon, left, sky, perched, shoulder, see, everything, better.)"
The Martyred Presidents,unknown,"List(film,, minute, long,, composed, two, shots., first,, girl, sits, base, altar, tomb,, face, hidden, camera., center, altar,, viewing, portal, displays, portraits, three, u.s., presidents—abraham, lincoln,, james, a., garfield,, william, mckinley—each, victims, assassination.)"
"Terrible Teddy, the Grizzly King",unknown,"List(""lasting, 61, seconds, consisting, two, shots,, first, shot, set, wood, winter., actor, representing, vice-president, theodore, roosevelt, enthusiastically, hurries, hillside, towards, tree, foreground., falls, once,, rights, cocks, rifle., two, men,, bearing, signs, reading, """"his, photographer"""", """"his, press, agent"""", respectively)"
Jack and the Beanstalk,unknown,"List(earliest, known, adaptation, classic, fairytale,, films, shows, jack, trading, cow, beans,, mother, forcing, drop, front, yard,, beig, forced, upstairs., sleeps,, jack, visited, fairy, shows, glimpses, await, ascends, bean, stalk., version,, jack, son, deposed, king., jack, wakes, up,, finds, beanstalk, grown, climbs, top, enters, giant's, home., giant, finds, jack,, narrowly, escapes., giant, chases, jack, bean, stalk,, jack, able, cut, giant, get, safety., falls, killed, jack, celebrates., fairy, reveals, jack, may, return, home, prince.)"
Alice in Wonderland,unknown,"List(""alice, follows, large, white, rabbit, """"rabbit-hole""""., finds, tiny, door., finds, bottle, labeled, """"drink, me"""")"
The Great Train Robbery,western,"List(film, opens, two, bandits, breaking, railroad, telegraph, office,, force, operator, gunpoint, train, stopped, transmit, orders, engineer, fill, locomotive's, tender, station's, water, tank., knock, operator, tie, up., train, stops, boarded, bandits‍—‌now, four., two, bandits, enter, express, car,, kill, messenger, open, box, valuables, dynamite;, others, kill, fireman, force, engineer, halt, train, disconnect, locomotive., bandits, force, passengers, train, rifle, belongings., one, passenger, tries, escape, instantly, shot, down., carrying, loot,, bandits, escape, locomotive,, later, stopping, valley, horses, left.)"
The Suburbanite,comedy,"List(film, family, move, suburbs,, hoping, quiet, life., things, start, go, wrong,, wife, gets, violent, starts, throwing, crockery,, leading, arrest.)"
The Little Train Robbery,unknown,"List(""the, opening, scene, shows, interior, robbers', den., walls, decorated, portraits, notorious, criminals, pictures, illustrating, exploits, famous, bandits., gang, lounging, about,, others, reading, novels, illustrated, papers., although, youthful, appearance,, dressed, like, typical, western, desperado., """"bandit, queen)"
where the remainder of the gang have been waiting for them. Believing they have successfully eluded their pursuers,abandoning everything,"List(, obliged, take, chances, foot., police, get, sight, fleeing, robbers, lively, chase, follows, tall, weeds)"


In [13]:
#TF
#hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(wordsData)
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", minDF=10.0)
model = cv.fit(wordsData)
vocabulary=model.vocabulary #we will use this to map back our words when using the recommender
featurizedData = model.transform(wordsData)

#IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

#TF-IDF
rescaledData = idfModel.transform(featurizedData)

display(rescaledData.select("Title", "words","features"))

Title,words,features
Kansas Saloon Smashers,"List(bartender, working, saloon,, serving, drinks, customers., fills, stereotypically, irish, man's, bucket, beer,, carrie, nation, followers, burst, inside., assault, irish, man,, pulling, hat, eyes, dumping, beer, head., group, begin, wrecking, bar,, smashing, fixtures,, mirrors,, breaking, cash, register., bartender, sprays, seltzer, water, nation's, face, group, policemen, appear, order, everybody, leave.[1])","List(0, 20749, List(80, 117, 121, 293, 355, 369, 763, 1090, 1163, 1277, 1325, 1407, 1918, 2075, 2135, 2856, 3241, 3376, 3490, 3854, 4343, 4647, 4884, 5092, 6735, 7041, 7345, 7922, 8225, 10926, 11738, 12676, 13125, 13421, 14515, 14968, 16205), List(7.666965822220131, 3.9812690352710836, 3.9943713421682725, 4.600636453296795, 4.775870808207695, 4.729504887649769, 5.3939419623272835, 5.614729799949034, 5.675354421765468, 5.7310433276260575, 5.771498282318371, 5.828117176317879, 6.132112824261183, 12.531288433771412, 6.221519411976768, 6.5020329949499365, 6.690085226452876, 6.66735697537532, 6.701646048853952, 6.8252600048211285, 6.9364856399313535, 7.11294207727291, 7.078455901201741, 7.243970339679314, 7.467113890993524, 7.788697515120986, 7.545075432463236, 7.6902574423077334, 15.31897156728196, 8.02049912917831, 8.02049912917831, 8.111470907384037, 8.16026107155347, 8.21155436594102, 8.322780001051244, 8.322780001051244, 8.5169360154922))"
Love by the Light of the Moon,"List(moon,, painted, smiling, face, hangs, park, night., young, couple, walking, past, fence, learn, railing, look, up., moon, smiles., embrace,, moon's, smile, gets, bigger., sit, bench, tree., moon's, view, blocked,, causing, frown., last, scene,, man, fans, woman, hat, moon, left, sky, perched, shoulder, see, everything, better.)","List(0, 20749, List(6, 8, 17, 46, 107, 119, 159, 174, 355, 380, 414, 452, 473, 524, 667, 933, 1079, 1527, 1955, 2625, 2640, 3739, 4055, 4106, 5092, 5765, 7524, 8071, 8137, 8369, 8408, 9134, 9244, 10018, 11112, 17202, 18182), List(2.7893905123237235, 2.9216937094112065, 3.145965816625726, 3.4837008894787047, 3.9615564940071257, 3.9982578608575534, 4.174987604385731, 4.279728733216694, 4.775870808207695, 4.775870808207695, 4.82271371696777, 4.886897061401198, 4.917668720067952, 5.033500535593074, 5.205350792519733, 5.448883080358584, 5.696407830963301, 5.888135186044132, 12.531288433771412, 6.419794896712965, 6.419794896712965, 6.773966710433578, 6.893313468066144, 6.981606075211823, 7.243970339679314, 7.2641730469968335, 7.518407185381074, 7.6902574423077334, 7.6006452836180465, 7.722006140622314, 7.629632820491299, 7.7547959634453045, 7.860156479103131, 7.860156479103131, 8.064950891749143, 17.516196144618178, 8.591043987645923))"
The Martyred Presidents,"List(film,, minute, long,, composed, two, shots., first,, girl, sits, base, altar, tomb,, face, hidden, camera., center, altar,, viewing, portal, displays, portraits, three, u.s., presidents—abraham, lincoln,, james, a., garfield,, william, mckinley—each, victims, assassination.)","List(0, 20749, List(3, 42, 47, 355, 438, 478, 641, 855, 888, 1380, 1530, 2163, 2257, 3083, 3086, 5311, 5596, 6005, 6600, 6812, 7406, 8655, 11771, 13966, 14126), List(2.62430570754077, 3.45388900505641, 3.522623695427876, 4.775870808207695, 5.020428454025721, 5.011807710981814, 5.15457846714631, 5.531975838920122, 5.438965643701239, 5.857675978559423, 5.898497973079678, 6.243338459371408, 6.311806258648868, 6.581282366604078, 6.64513383859061, 7.130641654372311, 7.185701431555338, 7.2641730469968335, 7.467113890993524, 7.418323726824092, 7.6006452836180465, 7.6902574423077334, 8.064950891749143, 8.265621587211296, 8.265621587211296))"
"Terrible Teddy, the Grizzly King","List(""lasting, 61, seconds, consisting, two, shots,, first, shot, set, wood, winter., actor, representing, vice-president, theodore, roosevelt, enthusiastically, hurries, hillside, towards, tree, foreground., falls, once,, rights, cocks, rifle., two, men,, bearing, signs, reading, 

#pySpark dataframe

Now that we have weights for all the words in all the movies, we need to prepare a dataframe for the pySpark ALS recommender. The dataframe for ALS has three columns: Users, Item and Rating. In our case Words represent Users, MOvies are the Items and the TF_IDF Weight are the Ratings. So we use the output from TF-IDF to build the dataframe with the three required columns.

To do this we use Koalas, which was introduced into Databricks in 2018. Koalas is a spark implementation of Pandas. We use this becouse of the ease of use of Pandas.

In [15]:
import numpy as np
import pandas as pd
import databricks.koalas as ks

We start by converting the output from TF-IDF into a Koalas dataframe

In [17]:
KrescaledData=ks.DataFrame(rescaledData)
display(KrescaledData.head(10))

,Title,Genre,words,rawFeatures,features
0,Kansas Saloon Smashers,unknown,"[bartender, working, saloon,, serving, drinks,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Love by the Light of the Moon,unknown,"[moon,, painted, smiling, face, hangs, park, n...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.7893905123237..."
2,The Martyred Presidents,unknown,"[film,, minute, long,, composed, two, shots., ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 2.62430570754077, 0.0, 0.0, 0...."
3,"Terrible Teddy, the Grizzly King",unknown,"[""lasting, 61, seconds, consisting, two, shots...","(0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 5.24861141508154, 0.0, 0.0, 0...."
4,Jack and the Beanstalk,unknown,"[earliest, known, adaptation, classic, fairyta...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,Alice in Wonderland,unknown,"[""alice, follows, large, white, rabbit, """"rabb...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,The Great Train Robbery,western,"[film, opens, two, bandits, breaking, railroad...","(0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 2.3838479058623605, 0.0, 5.2486114150815..."
7,The Suburbanite,comedy,"[film, family, move, suburbs,, hoping, quiet, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 2.7528651099791954, ..."
8,The Little Train Robbery,unknown,"[""the, opening, scene, shows, interior, robber...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,where the remainder of the gang have been wai...,abandoning everything,"[, obliged, take, chances, foot., police, get,...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.6622776660192281, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Same as we will use vocabulary to map back words, we create a list to map back movies

In [19]:
movieList = KrescaledData['Title'].tolist()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

We add the first movie to get the pySPark dataframe we need for ALS started.

In [21]:
thisMovie=KrescaledData['features'][0].toArray()

#get words for this movie
words=np.nonzero(thisMovie)

#get weights for the words in this movie
weights=thisMovie[thisMovie!=0]

#create an initial Pandas dataframe with the words and their weights for the first movie
matrixData = pd.DataFrame()
matrixData['word']= words[0]
matrixData['rating']=weights
matrixData['movie']=0

#convert Pandas dataframe to Koalas to make use of spark going forward
matrix=ks.DataFrame(matrixData)
#matrix

Now that we have a dataframe with one movie, we can add all the other movies in our dataset by following the same steps.

In [23]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [24]:
from multiprocessing.pool import ThreadPool

In [25]:
# allow up to 5 concurrent threads
pool = ThreadPool(5)

# hyperparameters to test out (n_trees)
parameters = [1,7]

# define a function to train a RF model and return metrics 
def mllib_random_forest(cluster,movies):
  thisMovie=KrescaledData['features'][cluster].toArray()
  #get words for this movie
  words=np.nonzero(thisMovie)
  #get weights for the words in this movie
  weights=thisMovie[thisMovie!=0]
  #create an initial Pandas dataframe with the words and their weights for the first movie
  matrixData = pd.DataFrame()
  matrixData['word']= words[0]
  matrixData['rating']=weights
  matrixData['movie']=0
  #convert Pandas dataframe to Koalas to make use of spark going forward
  matrix=ks.DataFrame(matrixData)  
  for i in range(cluster,cluster+5):
    thisMovie=movies[i].toArray()
    #get words for this movie
    words=np.nonzero(thisMovie)
    #get weights for the words in this movie
    weights=thisMovie[thisMovie!=0]
    matrixData = pd.DataFrame()
    matrixData['word']= words[0]
    matrixData['rating']=weights
    matrixData['movie']=i
    matrix=matrix.append(ks.DataFrame(matrixData))
  return [matrix]
  
# run the tasks 
pool.map(lambda cluster: mllib_random_forest(cluster,KrescaledData['features']), parameters)
 

Out[85]: [[ word rating movie
 0 6 2.789391 0
 1 8 2.921694 0
 2 17 3.145966 0
 3 46 3.483701 0
 4 107 3.961556 0
 5 119 3.998258 0
 6 159 4.174988 0
 7 174 4.279729 0
 8 355 4.775871 0
 9 380 4.775871 0
 10 413 4.822714 0
 11 452 4.886897 0
 12 473 4.917669 0
 13 523 5.033501 0
 14 667 5.205351 0
 15 933 5.448883 0
 16 1079 5.696408 0
 17 1533 5.888135 0
 18 1955 12.531288 0
 19 2622 6.419795 0
 20 2647 6.419795 0
 21 3743 6.773967 0
 22 4069 6.893313 0
 23 4120 6.981606 0
 24 5158 7.243970 0
 25 5726 7.264173 0
 26 7568 7.518407 0
 27 8160 7.690257 0
 28 8168 7.600645 0
 29 8300 7.629633 0
 30 8393 7.722006 0
 31 9112 7.754796 0
 32 9254 7.860156 0
 33 10040 7.860156 0
 34 11354 8.064951 0
 35 17181 17.516196 0
 36 18013 8.591044 0
 0 6 2.789391 1
 1 8 2.921694 1
 2 17 3.145966 1
 3 46 3.483701 1
 4 107 3.961556 1
 5 119 3.998258 1
 6 159 4.174988 1
 7 174 4.279729 1
 8 355 4.775871 1
 9 380 4.775871 1
 10 413 4.822714 1
 11 452 4.886897 1
 12 473 4.917669 1
 13 523 5.033501 1
 14 667 5.205351 1
 15 933 5.448883 1
 16 1079 5.696408 1
 17 1533 5.888135 1
 18 1955 12.531288 1
 19 2622 6.419795 1
 20 2647 6.419795 1
 21 3743 6.773967 1
 22 4069 6.893313 1
 23 4120 6.981606 1
 24 5158 7.243970 1
 25 5726 7.264173 1
 26 7568 7.518407 1
 27 8160 7.690257 1
 28 8168 7.600645 1
 29 8300 7.629633 1
 30 8393 7.722006 1
 31 9112 7.754796 1
 32 9254 7.860156 1
 33 10040 7.860156 1
 34 11354 8.064951 1
 35 17181 17.516196 1
 36 18013 8.591044 1
 0 3 2.624306 2
 1 42 3.453889 2
 2 47 3.522624 2
 3 355 4.775871 2
 4 438 5.020428 2
 5 478 5.011808 2
 6 641 5.154578 2
 7 855 5.531976 2
 8 888 5.438966 2
 9 1380 5.857676 2
 10 1535 5.898498 2
 11 2162 6.243338 2
 12 2256 6.311806 2
 13 3080 6.581282 2
 14 3083 6.645134 2
 15 5316 7.130642 2
 16 5578 7.185701 2
 17 6025 7.264173 2
 18 6619 7.467114 2
 19 6825 7.418324 2
 20 7304 7.600645 2
 21 8638 7.690257 2
 22 11498 8.064951 2
 23 14193 8.265622 2
 24 14243 8.265622 2
 0 3 5.248611 3
 1 36 3.278975 3
 2 38 3.352558 3
 3 81 3.755984 3
 4 327 4.679021 3
 5 333 4.703944 3
 6 1045 5.610816 3
 7 1188 5.667056 3
 8 1702 6.093398 3
 9 1720 6.044006 3
 10 1906 6.145358 3
 11 2889 6.571026 3
 12 2928 6.511602 3
 13 3404 6.667357 3
 14 5488 7.167009 3
 15 6330 7.442421 3
 16 6959 7.442421 3
 17 8056 7.600645 3
 18 9710 7.823789 3
 19 10137 8.265622 3
 20 11196 7.977940 3
 21 12338 8.211554 3
 22 13351 8.211554 3
 23 15092 8.322780 3
 24 15982 8.447943 3
 25 19183 8.671087 3
 26 20026 8.758098 3
 0 9 2.921428 4
 1 18 6.146564 4
 2 20 3.174602 4
 3 24 3.269912 4
 4 30 3.244669 4
 5 36 3.278975 4
 6 60 3.604806 4
 7 122 4.002942 4
 8 193 4.297428 4
 9 195 4.265384 4
 10 198 4.278697 4
 11 208 37.823105 4
 12 257 9.042720 4
 13 259 4.513507 4
 14 263 4.523992 4
 15 347 4.734371 4
 16 412 4.820939 4
 17 445 4.885005 4
 18 630 5.144726 4
 19 917 5.432408 4
 20 1117 5.642565 4
 21 1142 5.634532 4
 22 1191 5.683723 4
 23 1259 5.748822 4
 24 1454 17.872489 4
 25 1584 5.924885 4
 26 2206 6.273191 4
 27 2390 6.296181 4
 28 2401 6.351972 4
 29 2795 6.473862 4
 30 3464 6.725177 4
 31 3790 6.825260 4
 32 4095 14.191101 4
 33 4860 7.078456 4
 34 5315 7.130642 4
 35 5767 7.327352 4
 36 5919 7.371804 4
 37 5945 7.394793 4
 38 6335 7.327352 4
 39 7357 7.492432 4
 40 7450 7.492432 4
 41 7753 7.754796 4
 42 11553 16.423109 4
 43 12166 8.064951 4
 44 15804 8.383405 4
 45 17368 8.671087 4
 46 18447 8.591044 4
 47 19645 8.758098 4
 0 18 6.146564 5
 1 219 4.364772 5
 2 330 4.683647 5
 3 394 4.973908 5
 4 2733 6.446463 5
 5 3258 6.725177 5
 6 3584 7.112942 5
 7 3700 6.865534 5
 8 15210 8.383405 5], [ word rating movie
 0 5 2.752865 0
 1 11 3.057959 0
 2 16 3.102742 0
 3 17 3.145966 0
 4 44 3.410991 0
 5 70 3.737212 0
 6 132 4.048568 0
 7 229 4.447909 0
 8 279 4.571202 0
 9 342 4.708687 0
 10 367 4.727888 0
 11 911 5.432408 0
 12 1123 5.650662 0
 13 1843 6.068397 0
 14 2488 6.351972 0
 15 4482 6.966339 0
 16 10939 7.937118 0
 0 5 2.752865 7
 1 11 3.057959 7
 2 16 3.102742 7
 3 17 3.145966 7
 4 44 3.410991 7
 5 70 3.737212 7
 

In [26]:
for i in range(1,500):
  thisMovie=KrescaledData['features'][i].toArray()
  #get words for this movie
  words=np.nonzero(thisMovie)
  #get weights for the words in this movie
  weights=thisMovie[thisMovie!=0]
  matrixData = pd.DataFrame()
  matrixData['word']= words[0]
  matrixData['rating']=weights
  matrixData['movie']=i
  matrix=matrix.append(ks.DataFrame(matrixData))
#matrix

Convert results from Koalas to databricks

In [28]:
df_matrix=matrix.to_spark()
df_matrix.count()

Split data into training and test

In [30]:
(training,test)=df_matrix.randomSplit([0.8,0.2])

#print('Training: {0}, test: {1}\n'.format(
#  training.count(), test.count())
#)
#training.show(3)
#test.show(3)

#Model

Train ALS models with different K

In [32]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [33]:
als = ALS()

als.setUserCol('word')\
   .setItemCol('movie')

param_grid=ParamGridBuilder()\
            .addGrid(als.rank,[8])\
            .addGrid(als.maxIter,[10])\
            .addGrid(als.regParam,[.1])\
            .build()

evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

tvs=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)

In [34]:
import mlflow
import mlflow.sklearn

In [35]:
ALSmodel=tvs.fit(training)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [36]:
best_model=ALSmodel.bestModel

predictions=best_model.transform(test) 

predicted_ratings_df = predictions.filter(predictions.prediction != float('nan'))

rmse=evaluator.evaluate(predicted_ratings_df)

print("RMSE = " + str(rmse))
print("**Best Model**")
print(" Rank:"),best_model.rank
print(" MaxIter:"),best_model._java_obj.parent().getMaxIter()
print(" RegParam:"),best_model._java_obj.parent().getRegParam()

RMSE = 2.462024420342563
**Best Model**
 Rank:
 MaxIter:
 RegParam:
Out[63]: (None, 0.1)

In [37]:
display(predicted_ratings_df)

word,rating,movie,prediction
18296,8.591043987645923,1,8.526204
6,2.7893905123237235,1,2.768338
5167,7.243970339679314,1,7.15207
9299,7.860156479103131,1,7.800833
8,2.9216937094112065,1,2.8996422
4087,6.981606075211823,1,6.9289126
1078,5.696407830963301,1,5.6534147
119,3.9982578608575534,1,3.9680815
662,5.17205706442027,6,0.47961414
7066,7.4424212784031525,3,7.3923683


#Recommendations

We start by computing the 10 recommended movies for each word.

In [39]:
user_rec=best_model.recommendForAllUsers(10)

Recommendations are made for earch word. So for instance if we thing of a given word in the vocabulary, we would like to know what movies best fit such word. Lets say we want recommendations for word 7678 in the vocabulary.

In [41]:
vocabulary[1959]

Out[66]: 'moon'

In [42]:
user = user_rec.filter(user_rec.word==1959)
rec=user.select("recommendations.movie","recommendations.rating")
rec_items=rec.select("movie").toPandas().iloc[0,0]
rec_ratings=rec.select("rating").toPandas().iloc[0,0]
rec_matrix=pd.DataFrame(rec_items,columns=["movie"])
rec_matrix["ratings"]=rec_ratings
ratings_matrix_df=sqlContext.createDataFrame(rec_matrix)
display(ratings_matrix_df)

movie,ratings
1,12.43671
8,5.5398307
2,5.1620774
6,4.6677046
0,3.9043462
3,2.351719
7,2.1631382
9,-4.5959554
5,-5.635323
4,-6.1553035


We use the list of movie titles to find which movie the selected word recommends

In [44]:
movieList[ratings_matrix_df.select('movie').collect()[0].movie]

Out[68]: 'Love by the Light of the Moon'